In [1]:
# third-party
import pandas as pd
import numpy as np
import pickle

# local
from waveform import get_waveforms_by_id_by_activity
from get_data import get_data_by_id_activity
from constants import DEVICE_INFO

In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

acquisition_folderpath = 'Aquisicao'
data, _ = get_data_by_id_activity(acquisition_folderpath)

Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [3]:
amplitudes_df = pd.DataFrame(columns=['amplitude', 'activity', 'sensor', 'id'])

for id in overview.keys():
    for activity in overview[id].keys():
        for device in ['mag', 'pzt', 'airflow']:
            try:
                waveforms,_ = get_waveforms_by_id_by_activity(data[id][activity][device].values, overview[id][activity][DEVICE_INFO[device]["name"]])
                x = np.amax(waveforms, axis=1) - np.amin(waveforms, axis=1)
                amplitudes_df = pd.concat([amplitudes_df, pd.DataFrame({'amplitude': x, 'activity': activity, 'sensor': DEVICE_INFO[device]["sensor"], 'id': id})], ignore_index=True)
            except Exception as e:
                print(e)
                print(id, activity, device)

amplitudes_df            

zero-size array to reduction operation maximum which has no identity
83J1 UAL pzt


,amplitude,activity,sensor,id
0,33.384048,SNB,MAG,7OYX
1,36.360410,SNB,MAG,7OYX
2,62.917256,SNB,MAG,7OYX
3,36.316845,SNB,MAG,7OYX
4,35.134079,SNB,MAG,7OYX
...,...,...,...,...
4798,1573.308449,SS,PZT,D4GQ
4799,1371.299671,SS,Airflow,D4GQ
4800,1908.289219,SS,Airflow,D4GQ
4801,1256.888880,SS,Airflow,D4GQ


In [4]:
amplitudes_df[amplitudes_df["sensor"]=='MAG'].max()

amplitude    223.258456
activity            UAR
sensor              MAG
id                 Y6O3
dtype: object

In [5]:
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']):.1f} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']):.1f}")
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']):.1f} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']):.1f}")
print(f"pk-to-pk for Airflow sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']):.1f} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']):.1f}\n")

print(f"coefficient of variation MAG: {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']):.2f}")
print(f"coefficient of variation PZT: {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']):.2f}")
print(f"coefficient of variation Airflow: {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']):.2f}")

pk-to-pk for MAG sensor: 41.0 +/- 27.6
pk-to-pk for MAG sensor: 836.8 +/- 704.3
pk-to-pk for Airflow sensor: 856.1 +/- 516.8

coefficient of variation MAG: 0.67
coefficient of variation PZT: 0.84
coefficient of variation Airflow: 0.60


In [6]:
amplitudes_by_activity_df = pd.DataFrame(columns=['Activity', 'Sensor', 'Pk-to-pk amplitude', 'std/mean'])

for activity in amplitudes_df['activity'].unique():
    amps = amplitudes_df[(amplitudes_df['activity'] == activity) & (amplitudes_df['sensor'] == 'MAG')]['amplitude'].values
    mean_amp = np.mean(amps)
    std_amp = np.std(amps)
    amplitudes_by_activity_df = pd.concat([amplitudes_by_activity_df, pd.DataFrame({'Activity': activity, 'Sensor': 'MAG', 'Pk-to-pk amplitude': f'${np.around(mean_amp, 2)} \pm {np.around(std_amp)}$', 'std/mean': np.round(std_amp/mean_amp, 2)}, index=[0])], ignore_index=True)
    
    amps = amplitudes_df[(amplitudes_df['activity'] == activity) & (amplitudes_df['sensor'] == 'PZT')]['amplitude'].values
    mean_amp = np.mean(amps)
    std_amp = np.std(amps)
    amplitudes_by_activity_df = pd.concat([amplitudes_by_activity_df, pd.DataFrame({'Activity': activity, 'Sensor': 'PZT', 'Pk-to-pk amplitude': f'${np.around(mean_amp, 2)} \pm {np.around(std_amp)}$', 'std/mean': np.round(std_amp/mean_amp, 2)}, index=[0])], ignore_index=True)

amplitudes_by_activity_df

,Activity,Sensor,Pk-to-pk amplitude,std/mean
0,SNB,MAG,$29.03 \pm 15.0$,0.53
1,SNB,PZT,$795.39 \pm 555.0$,0.70
2,SGB,MAG,$80.05 \pm 38.0$,0.47
3,SGB,PZT,$1348.85 \pm 847.0$,0.63
4,MIXB,MAG,$41.88 \pm 21.0$,0.51
5,MIXB,PZT,$1221.2 \pm 712.0$,0.58
6,STNB,MAG,$35.49 \pm 21.0$,0.59
7,STNB,PZT,$863.16 \pm 548.0$,0.64
8,MCH,MAG,$38.56 \pm 21.0$,0.53
9,MCH,PZT,$846.84 \pm 521.0$,0.62


In [7]:
amplitudes_df

,amplitude,activity,sensor,id
0,33.384048,SNB,MAG,7OYX
1,36.360410,SNB,MAG,7OYX
2,62.917256,SNB,MAG,7OYX
3,36.316845,SNB,MAG,7OYX
4,35.134079,SNB,MAG,7OYX
...,...,...,...,...
4798,1573.308449,SS,PZT,D4GQ
4799,1371.299671,SS,Airflow,D4GQ
4800,1908.289219,SS,Airflow,D4GQ
4801,1256.888880,SS,Airflow,D4GQ


### Analysis on alternative positioning

In [8]:
amplitudes_middle_df = pd.DataFrame(columns=['amplitude', 'activity', 'sensor', 'id'])

for id in overview_middle.keys():
    for activity in overview_middle[id].keys():
        for device in ['mag', 'pzt']:
            waveforms,_ = get_waveforms_by_id_by_activity(data[id][activity][device].values, overview_middle[id][activity][DEVICE_INFO[device]["name"]])
            x = np.amax(waveforms, axis=1) - np.amin(waveforms, axis=1)
            amplitudes_middle_df = pd.concat([amplitudes_middle_df, pd.DataFrame({'amplitude': x, 'activity': activity, 'sensor': DEVICE_INFO[device]["sensor"], 'id': id})], ignore_index=True)

amplitudes_middle_df            

,amplitude,activity,sensor,id
0,18.016615,SNBm,MAG,7OYX
1,13.768694,SNBm,MAG,7OYX
2,20.512037,SNBm,MAG,7OYX
3,22.874771,SNBm,MAG,7OYX
4,31.799197,SNBm,MAG,7OYX
...,...,...,...,...
826,33.344231,UARm,MAG,D4GQ
827,83.290504,UARm,MAG,D4GQ
828,1609.729299,UARm,PZT,D4GQ
829,2142.694495,UARm,PZT,D4GQ


In [9]:
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG'])}")
print(f"pk-to-pk for PZT sensor: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='PZT'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='PZT'])}")
print(f"pk-to-pf for Airflow: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='Airflow'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='Airflow'])}")
print(np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG']) / np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG']))

pk-to-pk for MAG sensor: 42.8603020637808 +/- 27.514497955022033
pk-to-pk for PZT sensor: 664.3808751049123 +/- 637.3878780022178
pk-to-pf for Airflow: nan +/- nan
0.6419576304916718
